In [2]:
import pandas as pd

In [2]:
input_path = r'D:\BAPS\Projects\mmil\All_Subjects_My_Table_19Sep2025.csv'
df = pd.read_csv(input_path)

df.head(), df.columns.tolist()

(   subject_id visit  DXNORM  DXMCI  DXAD
 0  002_S_0295    bl     1.0    NaN   NaN
 1  002_S_0295   m06     1.0    NaN   NaN
 2  002_S_0295   m12     1.0    NaN   NaN
 3  002_S_0295   m18     NaN    NaN   NaN
 4  002_S_0295   m24     1.0    NaN   NaN,
 ['subject_id', 'visit', 'DXNORM', 'DXMCI', 'DXAD'])

In [3]:
print(df.shape)

(24501, 5)


In [4]:
df = df.dropna(subset=['DXNORM', 'DXMCI', 'DXAD'], how='all')

In [11]:
col = ['DXNORM', 'DXMCI', 'DXAD']
df[col] = df[col].fillna(0.0)

df[col] = df[col].astype(int)

df.head()

subject_id visit  DXNORM  DXMCI  DXAD
0  002_S_0295    bl       1      0     0
1  002_S_0295   m06       1      0     0
2  002_S_0295   m12       1      0     0
4  002_S_0295   m24       1      0     0
6  002_S_0295   m36       1      0     0

In [5]:
print(df.shape)

(3860, 5)


In [13]:
normal_count = df['DXNORM'].sum()
mci_count    = df['DXMCI'].sum()
ad_count     = df['DXAD'].sum()

print("Class distribution:")
print(f"Normal: {normal_count}")
print(f"MCI: {mci_count}")
print(f"AD: {ad_count}")

Class distribution:
Normal: 1130
MCI: 1601
AD: 1129


In [14]:
df['subject_id'].nunique()

819

In [15]:
output_file = "Processed_11.csv"
df.to_csv(output_file, index=False)

In [17]:
df_bl = df[df['visit'].str.lower() == 'bl']

In [18]:
df_bl

subject_id visit  DXNORM  DXMCI  DXAD
0      002_S_0295    bl       1      0     0
14     002_S_0413    bl       1      0     0
40     002_S_0559    bl       1      0     0
50     002_S_0619    bl       0      0     1
57     002_S_0685    bl       1      0     0
...           ...   ...     ...    ...   ...
24064  941_S_1202    bl       1      0     0
24080  941_S_1203    bl       1      0     0
24091  941_S_1295    bl       0      1     0
24097  941_S_1311    bl       0      1     0
24103  941_S_1363    bl       0      1     0

[819 rows x 5 columns]

In [20]:
normal_count = df_bl['DXNORM'].sum()
mci_count    = df_bl['DXMCI'].sum()
ad_count     = df_bl['DXAD'].sum()

print("Class distribution:")
print(f"Normal: {normal_count}")
print(f"MCI: {mci_count}")
print(f"AD: {ad_count}")

Class distribution:
Normal: 229
MCI: 397
AD: 193


In [21]:
df_geno = pd.read_csv(r'D:\BAPS\Projects\mmil\ADNI_1_GWAS_Plink\ADNI_cluster_01_forward_757LONI.fam', delim_whitespace=True)

In [22]:
df_geno

n         IID  a  b  c  d
0      1  014_S_0520  0  0  2 -9
1      2  005_S_1341  0  0  2 -9
2      3  012_S_1175  0  0  1 -9
3      4  012_S_0803  0  0  2 -9
4      5  018_S_0055  0  0  1 -9
..   ...         ... .. .. .. ..
752  813  013_S_0699  0  0  1 -9
753  814  057_S_0934  0  0  2 -9
754  815  141_S_1052  0  0  2 -9
755  816  129_S_0778  0  0  1 -9
756  817  053_S_0919  0  0  1 -9

[757 rows x 6 columns]

In [23]:
merged = pd.merge(df_bl, df_geno, left_on="subject_id", right_on="IID", how="inner")

print("Merged dataset shape:", merged.shape)
print("Number of matched subjects:", merged["IID"].nunique())

merged.to_csv("Processed_22.csv", index=False)

Merged dataset shape: (757, 11)
Number of matched subjects: 757


In [25]:
input_path = r'Processed_22.csv'
df = pd.read_csv(input_path)


normal_count = df['DXNORM'].sum()
mci_count    = df['DXMCI'].sum()
ad_count     = df['DXAD'].sum()

print("Class distribution:")
print(f"Normal: {normal_count}")
print(f"MCI: {mci_count}")
print(f"AD: {ad_count}")

Class distribution:
Normal: 214
MCI: 363
AD: 180


In [37]:
# Split into MCI and non-MCI
df_mci = df[df['DXMCI'] == 1]
df_non_mci = df[df['DXMCI'] == 0]

# Randomly sample 220 MCI subjects
df_mci_down = df_mci.sample(n=220, random_state=42)

# Combine back
df_downsampled = pd.concat([df_non_mci, df_mci_down]).reset_index(drop=True)

# Shuffle rows so classes mix randomly
df_downsampled = df_downsampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the new file
df_downsampled.to_csv("Processed_22_downsampled.csv", index=False)

In [38]:
print("Class distribution after downsampling:")
print("Normal:", (df_downsampled["DXNORM"] == 1).sum())
print("MCI:", (df_downsampled["DXMCI"] == 1).sum())
print("AD:", (df_downsampled["DXAD"] == 1).sum())

Class distribution after downsampling:
Normal: 214
MCI: 220
AD: 180


In [40]:
subject_ids = df_downsampled["subject_id"].astype(str).tolist()
subject_ids = ",".join(subject_ids)
print(subject_ids)

131_S_0497,141_S_0696,033_S_0516,031_S_0830,024_S_1307,041_S_0446,126_S_0891,002_S_0559,130_S_0969,014_S_0558,062_S_0768,128_S_0715,022_S_0129,128_S_0245,027_S_0404,136_S_0196,057_S_0779,021_S_0647,068_S_0127,036_S_0576,128_S_0205,136_S_0186,067_S_0257,116_S_0487,022_S_0004,041_S_0262,016_S_1121,057_S_0818,133_S_0493,041_S_1010,002_S_1280,029_S_0843,116_S_1249,027_S_0417,029_S_0999,136_S_0107,013_S_1120,137_S_0973,012_S_0637,016_S_1263,022_S_0130,133_S_1031,128_S_0216,018_S_0142,024_S_1171,052_S_1251,007_S_1248,137_S_0994,126_S_1077,052_S_1352,137_S_0158,057_S_0934,116_S_1232,041_S_0549,013_S_1205,031_S_0618,014_S_0548,057_S_1373,109_S_1114,068_S_0442,032_S_0095,006_S_0731,009_S_1199,137_S_1426,036_S_1135,141_S_0810,007_S_1222,027_S_0835,099_S_0090,037_S_0454,037_S_0501,100_S_1286,033_S_1281,027_S_1082,014_S_1095,016_S_0991,067_S_0243,013_S_0502,007_S_0101,014_S_0328,141_S_1245,073_S_1357,005_S_1224,041_S_0282,021_S_1109,128_S_0517,041_S_0721,035_S_0204,002_S_0938,109_S_1192,027_S_0120

In [33]:
pd.set_option('display.max_rows', None)  # show all rows
pd.set_option('display.max_columns', None)

In [34]:
with open("mylist.txt", "w") as f:
    f.write(", ".join(str(x) for x in list))


#######

$$$$$$

In [42]:
input_path = r'D:\BAPS\Projects\mmil\3_claasses\ADNI1_T1w_Cohort_at_m12_My_Table_23Sep2025.csv'
df = pd.read_csv(input_path)

df.head(), df.columns.tolist()

(   subject_id visit  DXNORM  DXMCI  DXAD
 0  002_S_0295    bl     1.0    NaN   NaN
 1  002_S_0295   m06     1.0    NaN   NaN
 2  002_S_0295   m12     1.0    NaN   NaN
 3  002_S_0295   m18     NaN    NaN   NaN
 4  002_S_0295   m24     1.0    NaN   NaN,
 ['subject_id', 'visit', 'DXNORM', 'DXMCI', 'DXAD'])

In [43]:
df = df.dropna(subset=['DXNORM', 'DXMCI', 'DXAD'], how='all')

In [47]:
df.head(), df.columns.tolist()

(   subject_id visit  DXNORM  DXMCI  DXAD
 0  002_S_0295    bl     1.0    NaN   NaN
 1  002_S_0295   m06     1.0    NaN   NaN
 2  002_S_0295   m12     1.0    NaN   NaN
 4  002_S_0295   m24     1.0    NaN   NaN
 6  002_S_0295   m36     1.0    NaN   NaN,
 ['subject_id', 'visit', 'DXNORM', 'DXMCI', 'DXAD'])

In [49]:
df.shape
df_m12.shape

(211, 5)

In [48]:
df_m12 = df[df['visit'].str.lower() == 'm12']

In [50]:
df_m12

subject_id visit  DXNORM  DXMCI  DXAD
2     002_S_0295   m12     1.0    NaN   NaN
17    002_S_0413   m12     1.0    NaN   NaN
40    002_S_0559   m12     1.0    NaN   NaN
49    002_S_0685   m12     1.0    NaN   NaN
68    002_S_1261   m12     1.0    NaN   NaN
87    002_S_1280   m12     1.0    NaN   NaN
109   003_S_0907   m12     1.0    NaN   NaN
126   003_S_0931   m12     1.0    NaN   NaN
136   003_S_0981   m12     1.0    NaN   NaN
155   005_S_0553   m12     1.0    NaN   NaN
174   005_S_0602   m12     1.0    NaN   NaN
194   005_S_0610   m12     1.0    NaN   NaN
214   006_S_0498   m12     1.0    NaN   NaN
234   006_S_0681   m12     1.0    NaN   NaN
242   006_S_0731   m12     1.0    NaN   NaN
262   007_S_0068   m12     1.0    NaN   NaN
272   007_S_0070   m12     1.0    NaN   NaN
279   007_S_1206   m12     1.0    NaN   NaN
293   007_S_1222   m12     1.0    NaN   NaN
313   009_S_0751   m12     1.0    NaN   NaN
337   009_S_0842   m12     1.0    NaN   NaN
355   009_S_0862   m12     1.0    NaN   NaN
364   010_S_0067   m12     1.0    NaN   NaN
373   010_S_0419   m12     1.0    NaN   NaN
392   010_S_0420   m12     1.0    NaN   NaN
411   010_S_0422   m12     1.0    NaN   NaN
420   010_S_0472   m12     1.0    NaN   NaN
427   011_S_0005   m12     1.0    NaN   NaN
437   011_S_0008   m12     1.0    NaN   NaN
453   011_S_0016   m12     1.0    NaN   NaN
462   011_S_0021   m12     1.0    NaN   NaN
484   011_S_0022   m12     1.0    NaN   NaN
491   011_S_0023   m12     1.0    NaN   NaN
510   012_S_0637   m12     1.0    NaN   NaN
526   012_S_1133   m12     1.0    NaN   NaN
540   012_S_1212   m12     1.0    NaN   NaN
555   013_S_0502   m12     1.0    NaN   NaN
563   013_S_0575   m12     1.0    NaN   NaN
571   013_S_1035   m12     1.0    NaN   NaN
579   013_S_1276   m12     1.0    NaN   NaN
587   014_S_0519   m12     1.0    NaN   NaN
603   014_S_0520   m12     1.0    NaN   NaN
619   014_S_0548   m12     1.0    NaN   NaN
635   014_S_0558   m12     1.0    NaN   NaN
651   016_S_0359   m12     1.0    NaN   NaN
667   016_S_0538   m12     1.0    NaN   NaN
679   018_S_0043   m12     1.0    NaN   NaN
690   018_S_0055   m12     1.0    NaN   NaN
705   018_S_0369   m12     1.0    NaN   NaN
713   018_S_0425   m12     1.0    NaN   NaN
719   020_S_0097   m12     1.0    NaN   NaN
727   020_S_0883   m12     1.0    NaN   NaN
735   020_S_0899   m12     1.0    NaN   NaN
743   020_S_1288   m12     1.0    NaN   NaN
754   021_S_0159   m12     1.0    NaN   NaN
778   021_S_0337   m12     1.0    NaN   NaN
803   021_S_0647   m12     1.0    NaN   NaN
812   021_S_0984   m12     1.0    NaN   NaN
829   022_S_0066   m12     1.0    NaN   NaN
837   022_S_0096   m12     1.0    NaN   NaN
861   022_S_0130   m12     1.0    NaN   NaN
877   023_S_0031   m12     1.0    NaN   NaN
899   023_S_0058   m12     1.0    NaN   NaN
918   023_S_0061   m12     1.0    NaN   NaN
937   023_S_0081   m12     1.0    NaN   NaN
949   023_S_0926   m12     1.0    NaN   NaN
969   023_S_0963   m12     1.0    NaN   NaN
979   023_S_1190   m12     1.0    NaN   NaN
1002  024_S_0985   m12     1.0    NaN   NaN
1018  024_S_1063   m12     1.0    NaN   NaN
1030  027_S_0074   m12     1.0    NaN   NaN
1056  027_S_0118   m12     1.0    NaN   NaN
1073  027_S_0120   m12     1.0    NaN   NaN
1093  027_S_0403   m12     1.0    NaN   NaN
1105  029_S_0824   m12     1.0    NaN   NaN
1124  029_S_0843   m12     1.0    NaN   NaN
1141  029_S_0845   m12     1.0    NaN   NaN
1152  029_S_0866   m12     1.0    NaN   NaN
1161  031_S_0618   m12     1.0    NaN   NaN
1183  032_S_0095   m12     1.0    NaN   NaN
1190  032_S_0479   m12     1.0    NaN   NaN
1205  032_S_0677   m12     1.0    NaN   NaN
1225  032_S_1169   m12     1.0    NaN   NaN
1243  033_S_0516   m12     1.0    NaN   NaN
1254  033_S_0734   m12     1.0    NaN   NaN
1279  033_S_0741   m12     1.0    NaN   NaN
1300  033_S_0920   m12     1.0    NaN   NaN
1319  033_S_0923   m12     1.0    NaN   NaN
1339  033_S_1016   m12     1.0    NaN   NaN
1363  033_S_1086   m12     1.0    NaN   NaN
13

In [51]:
col = ['DXNORM', 'DXMCI', 'DXAD']
df_m12[col] = df_m12[col].fillna(0.0)

df_m12[col] = df_m12[col].astype(int)

df_m12.head()

C:\Users\BAPS\AppData\Local\Temp\ipykernel_20056\2995329746.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m12[col] = df_m12[col].fillna(0.0)
C:\Users\BAPS\AppData\Local\Temp\ipykernel_20056\2995329746.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m12[col] = df_m12[col].astype(int)


subject_id visit  DXNORM  DXMCI  DXAD
2   002_S_0295   m12       1      0     0
17  002_S_0413   m12       1      0     0
40  002_S_0559   m12       1      0     0
49  002_S_0685   m12       1      0     0
68  002_S_1261   m12       1      0     0

In [52]:
output_file = "normal_.csv"
df_m12.to_csv(output_file, index=False)

In [55]:
input_path = r'D:\BAPS\Projects\mmil\3_claasses\ADNI1_T1w_Cohort_at_Visit_12_My_Table_23Sep2025.csv'
df = pd.read_csv(input_path)
df = df.dropna(subset=['DXNORM', 'DXMCI', 'DXAD'], how='all')
df_ad = df[df['visit'].str.lower() == 'm12']
col = ['DXNORM', 'DXMCI', 'DXAD']
df_ad[col] = df_ad[col].fillna(0.0)

df_ad[col] = df_ad[col].astype(int)

df_ad.head()
df_ad.shape

C:\Users\BAPS\AppData\Local\Temp\ipykernel_20056\111050959.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ad[col] = df_ad[col].fillna(0.0)
C:\Users\BAPS\AppData\Local\Temp\ipykernel_20056\111050959.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ad[col] = df_ad[col].astype(int)


(204, 5)

In [56]:
df_ad.head()

subject_id visit  DXNORM  DXMCI  DXAD
2   002_S_0619   m12       0      0     1
9   002_S_0729   m12       0      0     1
26  002_S_0816   m12       0      0     1
30  002_S_0938   m12       0      0     1
36  002_S_0954   m12       0      0     1

In [57]:
output_file = "AD_.csv"
df_ad.to_csv(output_file, index=False)

In [58]:
input_path = r'D:\BAPS\Projects\mmil\3_claasses\ADNI1_T1w_DXMCI=1_at_m18_My_Table_23Sep2025.csv'
df = pd.read_csv(input_path)
df = df.dropna(subset=['DXNORM', 'DXMCI', 'DXAD'], how='all')
df_mci = df[df['visit'].str.lower() == 'm18']
col = ['DXNORM', 'DXMCI', 'DXAD']
df_mci[col] = df_mci[col].fillna(0.0)

df_mci[col] = df_mci[col].astype(int)
df_mci.shape

C:\Users\BAPS\AppData\Local\Temp\ipykernel_20056\4060520090.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mci[col] = df_mci[col].fillna(0.0)
C:\Users\BAPS\AppData\Local\Temp\ipykernel_20056\4060520090.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mci[col] = df_mci[col].astype(int)


(212, 5)

In [59]:
df_mci.head()

subject_id visit  DXNORM  DXMCI  DXAD
3   002_S_0782   m18       0      1     0
16  002_S_1155   m18       0      1     0
34  002_S_1268   m18       0      1     0
53  003_S_0908   m18       0      1     0
77  003_S_1074   m18       0      1     0

In [60]:
output_file = "mci_.csv"
df_mci.to_csv(output_file, index=False)

In [62]:
df1 = pd.read_csv("normal_.csv")
df2 = pd.read_csv("mci_.csv")
df3 = pd.read_csv("AD_.csv")

combined_df = pd.concat([df1, df2, df3], ignore_index=True)

print(combined_df.shape) 
print(combined_df.head())

combined_df.to_csv("combined.csv", index=False)

(627, 5)
   subject_id visit  DXNORM  DXMCI  DXAD
0  002_S_0295   m12       1      0     0
1  002_S_0413   m12       1      0     0
2  002_S_0559   m12       1      0     0
3  002_S_0685   m12       1      0     0
4  002_S_1261   m12       1      0     0


In [64]:
df = pd.read_csv("combined.csv")

In [2]:
df_geno = pd.read_csv('D:\BAPS\Projects\mmil\ADNI_1_GWAS_Plink\ADNI_cluster_01_forward_757LONI.fam', delim_whitespace=True)

In [65]:
merged = pd.merge(df, df_geno, left_on="subject_id", right_on="IID", how="inner")

print("Merged dataset shape:", merged.shape)
print("Number of matched subjects:", merged["IID"].nunique())

# Save merged dataset
merged.to_csv("Processed_22.csv", index=False)

Merged dataset shape: (587, 11)
Number of matched subjects: 585


In [66]:
print("Class distribution after downsampling:")
print("Normal:", (merged["DXNORM"] == 1).sum())
print("MCI:", (merged["DXMCI"] == 1).sum())
print("AD:", (merged["DXAD"] == 1).sum())

Class distribution after downsampling:
Normal: 199
MCI: 198
AD: 190


In [3]:
df_m = pd.read_csv("Processed_gene_mri_subject_id.csv")

In [4]:
subject_ids = df_m["subject_id"].astype(str).tolist()

In [ ]:
# df = pd.read_csv('ADNI_genotypes.raw', delim_whitespace=True)
# #df = pd.read_csv(r"D:\plink_win64_20250819\ADNI_genotypes.raw", sep="\s+", engine="c")

raw_file = r"ADNI_genotypes.raw"
output_file = r"D:\plink_win64_20250819\ADNI_genotypes_filtered.csv"


filtered_chunks = []


chunk_iter = pd.read_csv(raw_file, delim_whitespace=True, chunksize=100)

for chunk in chunk_iter:
    # Keep only rows whose IID matches your list
    filtered_chunk = chunk[chunk["IID"].astype(str).isin(subject_ids)]
    if not filtered_chunk.empty:
        filtered_chunks.append(filtered_chunk)

# Combine into one DataFrame
df_filtered = pd.concat(filtered_chunks, ignore_index=True)

df_filtered.to_csv(output_file, index=False)

In [ ]:
print(f"Filtered data saved to {output_file}")
print("Final shape:", df_filtered.shape)

In [81]:
df = pd.read_csv(r'D:\plink_win64_20250819\ADNI_genotypes.raw', delim_whitespace=True, nrows=1)


['FID', 'IID', 'PAT', 'MAT', 'SEX', 'PHENOTYPE', 'rs12354060_0', 'rs2691310_0', 'rs2531266_0', 'rs4124251_0', 'rs8179466_0', 'rs6603779_0', 'cnvi0007379_0', 'cnvi0019140_0', 'cnvi0007389_0', 'cnvi0021559_0', 'cnvi0007392_0', 'cnvi0007380_0', 'cnvi0021553_0', 'cnvi0007381_0']


In [88]:
df.columns = df.columns.str.strip() 

In [93]:
print("IID" in df.columns)   # should be True

False


In [90]:
df_raw_filtered = df[df["IID"].astype(str).isin(subject_ids)]

KeyError: 'IID'

In [6]:
import pandas as pd

# Load your data
df = pd.read_csv("combined.csv")
df_geno = pd.read_csv(
    r'D:\BAPS\Projects\mmil\ADNI_1_GWAS_plink\ADNI_cluster_01_forward_757LONI.fam',
    delim_whitespace=True
)

# Merge (as before)
merged = pd.merge(df, df_geno, left_on="subject_id", right_on="IID", how="inner")

print("Merged dataset shape:", merged.shape)
print("Number of matched subjects:", merged["IID"].nunique())

# ---- NEW PART: find unmatched IDs ----
# Get all IIDs from genotype
all_iids = set(df_geno["IID"].astype(str))

# Get matched IIDs
matched_iids = set(merged["IID"].astype(str))

# Find unmatched IIDs
unmatched_iids = all_iids - matched_iids

print("Number of unmatched IIDs:", len(unmatched_iids))

# Save unmatched list into CSV
unmatched_df = pd.DataFrame({"Unmatched_IID": list(unmatched_iids)})
unmatched_df.to_csv("unmatched_iids.csv", index=False)

print("Unmatched IIDs saved to unmatched_iids.csv")


Merged dataset shape: (587, 11)
Number of matched subjects: 585
Number of unmatched IIDs: 172
Unmatched IIDs saved to unmatched_iids.csv
